# Grid mapping

cfgrib does not appear to parse the CF "grid_mapping" attribute.



In [5]:
from herbie.archive import Herbie
import metpy

In [2]:
H = Herbie('2021-9-21')

/p/home/blaylock/BB_python/Herbie/herbie/archive.py:205: UserWarning: `product` not specified. Will use ["sfc"].
  warnings.warn(f'`product` not specified. Will use ["{self.product}"].')


🏋🏻‍♂️ Found 2021-Sep-21 00:00 UTC F00 [HRRR] [product=sfc] GRIB2 file from aws and index file from aws.                                                                                                                                                       


In [3]:
ds = H.xarray('TMP:2 m')

In [7]:
ds.metpy.parse_cf().metpy_crs.item().to_cartopy()

In [24]:
grid_mapping = f'{ds.model}_projection'
ds[grid_mapping] = None
ds[grid_mapping].attrs['long_name'] = f"{ds.model.upper()} model grid projection"


In [25]:
var = list(ds)[0]

In [37]:
attrs = ds[var].attrs
if attrs['GRIB_gridType'] == 'lambert':
    ds[grid_mapping].attrs['grid_mapping_name'] = 'lambert_conformal_conic'
    ds[grid_mapping].attrs['standard_parallel'] = (attrs.get("GRIB_Latin1InDegrees"), attrs.get("GRIB_Latin2InDegrees"))
    ds[grid_mapping].attrs['longitude_of_central_meridian'] = attrs.get("GRIB_LoVInDegrees")
    ds[grid_mapping].attrs['latitude_of_projection_origin'] = attrs.get("GRIB_LaDInDegrees")
    ds[grid_mapping].attrs['false_easting'] = 0
    ds[grid_mapping].attrs['false_northing'] = 0
    
elif attrs['GRIB_gridType'] == 'polar_stereographic':
    ds[grid_mapping].attrs['grid_mapping_name'] = 'polar_stereographic'
    ds[grid_mapping].attrs['straight_vertical_longitude_from_pole'] = "not enough info"
    ds[grid_mapping].attrs['latitude_of_projection_origin'] = "not enough info (either +90 or -90)"
    ds[grid_mapping].attrs['standard_parallel'] = "not enough info"
    ds[grid_mapping].attrs['false_easting'] = 0
    ds[grid_mapping].attrs['false_northing'] = 0

elif attrs['GRIB_gridType'] == 'regular_ll':
    ds[grid_mapping].attrs['grid_mapping_name'] = 'latitude_longitude'

else:
    warnings.warn(f'GRIB_gridType [{attrs["GRIB_gridType"]}] is not recognized.')
                  
for v in list(ds):
    ds[v].attrs['grid_mapping'] = grid_mapping


In [39]:
import metpy

In [41]:
ds.metpy.parse_cf().metpy_crs.item().to_cartopy()

In [43]:
ds.metpy.parse_cf().metpy_crs.item().to_cartopy().proj4_params

{'ellps': 'sphere',
 'proj': 'lcc',
 'lon_0': 262.5,
 'lat_0': 38.5,
 'x_0': 0,
 'y_0': 0,
 'lat_1': 38.5,
 'lat_2': 38.5}

In [45]:
ds.herbie.crs.proj4_params

{'ellps': 'sphere',
 'proj': 'lcc',
 'lon_0': 262.5,
 'lat_0': 38.5,
 'x_0': 0.0,
 'y_0': 0.0,
 'lat_1': 38.5,
 'lat_2': 38.5}

In [27]:
Hak = Herbie('2021-09-21', model='hrrrak').xarray('TMP:2 m')
Hak

/p/home/blaylock/BB_python/Herbie/herbie/archive.py:205: UserWarning: `product` not specified. Will use ["prs"].
  warnings.warn(f'`product` not specified. Will use ["{self.product}"].')


🏋🏻‍♂️ Found 2021-Sep-21 00:00 UTC F00 [HRRRAK] [product=prs] GRIB2 file from nomads and index file from nomads.                                                                                                                                                       
👨🏻‍🏭 Created directory: [/p/cwfs/blaylock/data/hrrrak/20210921]
📇 Download subset: [HRRRAK] model [prs] product run at 2021-Sep-21 00:00 UTC F00                                                            
 cURL from https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20210921/alaska/hrrr.t00z.wrfprsf00.ak.grib2
   1: GRIB_message=616 TMP:2 m above ground:anl


<xarray.Dataset>
Dimensions:            (y: 919, x: 1299)
Coordinates:
    time               datetime64[ns] 2021-09-21
    step               timedelta64[ns] 00:00:00
    heightAboveGround  float64 2.0
    latitude           (y, x) float64 41.61 41.63 41.64 ... 76.36 76.35 76.34
    longitude          (y, x) float64 185.1 185.1 185.2 ... 244.0 244.1 244.2
    valid_time         datetime64[ns] 2021-09-21
Dimensions without coordinates: y, x
Data variables:
    t2m                (y, x) float32 290.1 290.1 290.1 ... 269.3 268.4 268.5
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    model:                   hrrrak
    product:                 prs
    description:             High-Resolution Rapid Refresh - Alaska
    remote_grib:             https://nomads.ncep.noaa.gov/pub/data/nccf/com/h...
    local_grib:              /p/cwfs/blaylock/data/hrrrak/20210921/hrrr.t00z....

In [28]:
Hak.t2m

<xarray.DataArray 't2m' (y: 919, x: 1299)>
array([[290.08017, 290.08017, 290.08017, ..., 285.08017, 285.08017,
        285.08017],
       [290.08017, 290.14267, 290.08017, ..., 285.14267, 285.14267,
        285.08017],
       [290.14267, 290.14267, 290.08017, ..., 285.20517, 285.14267,
        285.08017],
       ...,
       [278.58017, 279.14267, 279.14267, ..., 270.20517, 270.58017,
        268.89267],
       [278.70517, 279.20517, 279.26767, ..., 270.64267, 269.20517,
        268.76767],
       [278.76767, 278.83017, 278.76767, ..., 269.33017, 268.39267,
        268.51767]], dtype=float32)
Coordinates:
    time               datetime64[ns] 2021-09-21
    step               timedelta64[ns] 00:00:00
    heightAboveGround  float64 2.0
    latitude           (y, x) float64 41.61 41.63 41.64 ... 76.36 76.35 76.34
    longitude          (y, x) float64 185.1 185.1 185.2 ... 244.0 244.1 244.2
    valid_time         datetime64[ns] 2021-09-21
Dimensions without coordinates: y, x
Attributes: (12/21)
    GRIB_paramId:                    167
    GRIB_dataType:                   fc
    GRIB_numberOfPoints:             1193781
    GRIB_typeOfLevel:                heightAboveGround
    GRIB_stepUnits:                  1
    GRIB_stepType:                   instant
    ...                              ...
    GRIB_shortName:                  2t
    GRIB_stepRange:                  0
    GRIB_units:                      K
    long_name:                       2 metre temperature
    units:                           K
    standard_name:                   air_temperature

In [30]:
Hgfs = Herbie('2021-09-21', model='gfs').xarray('TMP:2 m above')
Hgfs

/p/home/blaylock/BB_python/Herbie/herbie/archive.py:205: UserWarning: `product` not specified. Will use ["pgrb2.0p25"].
  warnings.warn(f'`product` not specified. Will use ["{self.product}"].')


🏋🏻‍♂️ Found 2021-Sep-21 00:00 UTC F00 [GFS] [product=pgrb2.0p25] GRIB2 file from aws and index file from aws.                                                                                                                                                       
📇 Download subset: [GFS] model [pgrb2.0p25] product run at 2021-Sep-21 00:00 UTC F00                                                            
 cURL from https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20210921/00/atmos/gfs.t00z.pgrb2.0p25.f000
   1: GRIB_message=580 TMP:2 m above ground:anl
   2: GRIB_message=584 APTMP:2 m above ground:anl


<xarray.Dataset>
Dimensions:            (latitude: 721, longitude: 1440)
Coordinates:
    time               datetime64[ns] 2021-09-21
    step               timedelta64[ns] 00:00:00
    heightAboveGround  float64 2.0
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time         datetime64[ns] 2021-09-21
Data variables:
    t2m                (latitude, longitude) float32 263.6 263.6 ... 222.0 222.0
    aptmp              (latitude, longitude) float32 257.3 257.3 ... 205.0 205.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    model:                   gfs
    product:                 pgrb2.0p25
    description:             Global Forecast System
    remote_grib:             https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20...
    local_grib:              /p/cwfs/blaylock/data/gfs/20210921/gfs.t00z.pgrb...

In [31]:
Hgfs.t2m

<xarray.DataArray 't2m' (latitude: 721, longitude: 1440)>
array([[263.61932, 263.61932, 263.61932, ..., 263.61932, 263.61932,
        263.61932],
       [263.81934, 263.81934, 263.91934, ..., 263.81934, 263.81934,
        263.81934],
       [264.21933, 264.21933, 264.21933, ..., 264.31934, 264.31934,
        264.31934],
       ...,
       [224.11934, 224.11934, 224.11934, ..., 224.11934, 224.11934,
        224.11934],
       [223.71933, 223.71933, 223.71933, ..., 223.61934, 223.61934,
        223.61934],
       [222.01933, 222.01933, 222.01933, ..., 222.01933, 222.01933,
        222.01933]], dtype=float32)
Coordinates:
    time               datetime64[ns] 2021-09-21
    step               timedelta64[ns] 00:00:00
    heightAboveGround  float64 2.0
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time         datetime64[ns] 2021-09-21
Attributes: (12/32)
    GRIB_paramId:                             167
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1038240
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_shortName:                           2t
    GRIB_stepRange:                           0
    GRIB_units:                               K
    long_name:                                2 metre temperature
    units:                                    K
    standard_name:                            air_temperature